# Question 3: How do choices about data such as i) amount of training data and ii) balance of classes in a classification problem affect the performance of a neural network?

## Introduction

### Overview

In this final notebook we will look at how choices about the data affect the Neural Network's (NN) performance. I assume you have read and understood both the Question 1 and Question 2 notebooks before starting this one.

The first part of this will focus on chaning the ratio between the training, validation, and testing sets. In Question 2 I used a 64/16/20 split but here we will see what happens with a much smaller or larger training set. The second part will then focus on adjusting the weighting of the different classes to force the model to pay more or less attention to certain classes.

In order to do this easily, I have created a function stored in functions.py that wraps all the code from Question 2. It takes the following inputs:
- features
    - The feature data
- labels
    - The label data
- test_ratio=0.2
    - The amount of data used for testing
- val_ratio=0.2
    - The amount of data used for validation
- hidden_size=64
    - The side of the hidden layer
- num_epochs=20
    - The number of epochs
- batch_size=32
    - The batch size
- learning_rate=0.001
    - The learning rate
- use_class_weights=False
    - Whether or not to balance the classes so they have an equal affect on the loss

## Imports

In [7]:
import pandas as pd

## Initialising the Dataset

Let us load the same dataset used in question 2.

In [8]:
# Change the path to where the csv file is stored on your pc
path = "/Users/Ryan/Documents/Ryan's Documents/Uni/Year 4/Comp/Coursework 2/SDSS.csv"
data = pd.read_csv(path)
features = data[['u', 'g', 'r', 'i', 'z']]
labels = data['class']
features

,u,g,r,i,z
0,19.47406,17.04240,15.94699,15.50342,15.22531
1,18.66280,17.21449,16.67637,16.48922,16.39150
2,19.38298,18.19169,17.47428,17.08732,16.80125
3,17.76536,16.60272,16.16116,15.98233,15.90438
4,17.55025,16.26342,16.43869,16.55492,16.61326
...,...,...,...,...,...
9995,18.81777,17.47053,16.91508,16.68305,16.50570
9996,18.27255,17.43849,17.07692,16.71661,16.69897
9997,18.75818,17.77784,17.51872,17.43302,17.42048
9998,18.88287,17.91068,17.53152,17.36284,17.13988


## Loading the train_sdss_model function

In [9]:
import os, sys
sys.path.append('../')
from functions import train_sdss_model

## Adjusting the Training/Validation/Testing ratios

Now we will call train_sdss_model function for different ratios.

In [ ]:
# Smaller training data (larger test_ratio, smaller train_ratio)
results_small_train = train_sdss_model(features, labels, 
                                       test_ratio=0.4, val_ratio=0.2, 
                                       num_epochs=20)

# Larger training data (smaller test_ratio, larger train_ratio)
results_large_train = train_sdss_model(features, labels, 
                                       test_ratio=0.2, val_ratio=0.2, 
                                       num_epochs=20)